# 보호구역 좌표 반경 판별 및 병합 검증 정리

서울시 교통사고 정보에 대해, **사고지점 반경 300m 이내에 보호구역(어린이/노인/장애인) 존재여부**를 시각화하여 직접 확인할 수 있도록 제작한 노트북입니다.


In [1]:
import pandas as pd
import numpy as np
import folium
from folium import Circle
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from math import radians, cos, sin, sqrt, asin, atan2

In [3]:

# 데이터 로드
accident_df = pd.read_csv("../../data/raw/all_accident_info_2023.csv")  # TAAS 사고데이터 
zone_df = pd.read_csv("../../data/external/protection_zone_data.csv")   # 보호구역 데이터


In [4]:

# 테스트용 상위 10개 사고지점 추출
sample_accidents = accident_df[['legaldong_name', 'route_nm','lat', 'lng']].head(10)
# 보호구역 위치 필터링
child_zone = zone_df[zone_df['보호구역종류'] == '어린이']
elderly_zone = zone_df[zone_df['보호구역종류'] == '노인']
disabled_zone = zone_df[zone_df['보호구역종류'] == '장애인']
sample_accidents    # 사고지점 좌표 확인


,legaldong_name,route_nm,lat,lng
0,서울특별시 강남구 논현동,논현로,37.514099,127.030505
1,서울특별시 강남구 역삼동,도곡로,37.493036,127.042572
2,서울특별시 강남구 신사동,강남대로,37.520495,127.018526
3,서울특별시 강남구 세곡동,헌릉로,37.464951,127.094760
4,서울특별시 강남구 신사동,논현로,37.524646,127.028470
5,서울특별시 강남구 대치동,선릉로,37.498498,127.051770
6,서울특별시 강남구 역삼동,논현로,37.496546,127.038885
7,서울특별시 강남구 도곡동,도곡로,37.489870,127.032716
8,서울특별시 강남구 수서동,광평로51길,37.490995,127.103680
9,서울특별시 강남구 수서동,광평로56길,37.487156,127.104119


## 1. 보호구역 지도 확인 (haversine 방식)
* 지도에 각 보호구역의 좌표를 마커로 표시
    * 어린이보호구역 - 파란색 / 노인보호구역 - 빨간색 / 장애인보호구역 - 초록색 
* 지도에 사고지점 좌표를 마커(검정색)로 표시하고, 사고지점좌표로부터 300m이내의 반경을 원으로 표시
    * 이 때, 좌표로부터 300m 반경에 존재하는 거리를 계산할 때 `haversine`방식을 사용
    * 단순 평면거리가 아니라 지구처럼 둥근 표면위에서의 위/경도 좌표사이의 실제거리를 계산할 때는 haversine방식이 필요하기 때문


In [16]:
# 거리 계산 함수 (하버사인 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (m)
    dlat = radians(lat2 - lat1) # 위도차이를 라디안으로 변환
    dlon = radians(lon2 - lon1) # 경도차이를 라디안으로 변환
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2 # haversine 공식
    c = 2 * asin(sqrt(a)) # 대원거리 계산
    return R * c # 거리(m) 변환

# 지도 중심 설정
m = folium.Map(location=[sample_accidents['lat'].mean(), sample_accidents['lng'].mean()], zoom_start=14)

# 사고지점 + 300m 원
for idx, row in sample_accidents.iterrows():
    lat, lng = row['lat'], row['lng']
    folium.Marker(
        location=[lat, lng],
        tooltip=f"사고지점 {idx+1}",
        icon=folium.Icon(color='black', icon='info-sign')
    ).add_to(m)
    Circle([lat, lng], radius=300, color='black', fill=False).add_to(m)

    # 보호구역 종류별로 사고지점 기준 필터링
    for zone_type, color in zip(["어린이", "노인", "장애인"], ["blue", "red", "green"]):
        sub_zones = zone_df[zone_df["보호구역종류"] == zone_type]

        for _, z in sub_zones.iterrows():
            distance = haversine(lat, lng, z["Y"], z["X"])
            x, y = z["X"], z["Y"]
            if distance <= 300:
                folium.CircleMarker(
                    location=[z["Y"], z["X"]],
                    radius=4,
                    color=color,
                    fill=True,
                    fill_opacity=0.7,
                    tooltip=f"{zone_type} 보호구역 ({int(distance)}m) x: {x}, y: {y}"
                ).add_to(m)

# 결과 출력
m

## 2. 보호구역 지도 확인 (KDTree 방식)

- `00_merge_protection_zone.py` 스크립트에서는  
  사고지점 반경 300m 이내에 보호구역이 존재하는지를 판별하기 위해  
  **KDTree 모델**을 활용한 거리 기반 탐색 로직이 사용됨
  
- 따라서 해당 KDTree 기반 로직과 동일한 방식으로  
  **사고지점 주변 보호구역을 탐색하고 지도에 시각적으로 표시**함

In [17]:

# 위경도를 라디안으로 변환
def latlng_to_radians(df, lat_col="lat", lng_col="lng"):
    return np.radians(df[[lat_col, lng_col]].values)

# KDTree 구축
def build_kdtree(df, lat_col="Y", lng_col="X"):
    coords_rad = latlng_to_radians(df, lat_col, lng_col)
    return KDTree(coords_rad), coords_rad

# 라디안 반경 계산
EARTH_RADIUS_M = 6371000
radius_m = 300
radius_rad = radius_m / EARTH_RADIUS_M

# 지도 생성
m = folium.Map(location=[sample_accidents['lat'].mean(), sample_accidents['lng'].mean()], zoom_start=14)

# 사고지점 마커 + 원 표시 및 반경 내 보호구역 KDTree 탐색
for idx, row in sample_accidents.iterrows():
    lat, lng = row['lat'], row['lng']
    folium.Marker(
        location=[lat, lng], 
        tooltip=f"사고지점 {idx+1}",
        icon=folium.Icon(color='black')
    ).add_to(m)
    Circle([lat, lng], radius=300, color='black', fill=False).add_to(m)

    # 사고 좌표 라디안
    acc_rad = np.radians([lat, lng])
    # acc_rad = np.radians([[lat, lng]])  # (1, 2) shape

    for zone_type, color in zip(["어린이", "노인", "장애인"], ["blue", "red", "green"]):
        sub_zone = zone_df[zone_df['보호구역종류'] == zone_type]
        tree, coords_rad = build_kdtree(sub_zone, "Y", "X")

        # KDTree로 반경 내 보호구역 인덱스 찾기
        indices = tree.query_ball_point(acc_rad, r=radius_rad)
        # indices = tree.query_ball_point(acc_rad[0], r=radius_rad)

        for i in indices:
            row = sub_zone.iloc[i]
            folium.CircleMarker(
                location=[row["Y"], row["X"]],
                radius=4,
                color=color,
                fill=True,
                fill_opacity=0.7,
                tooltip=f"{zone_type} 보호구역 x: {row['X']}, y: {row['Y']}",
            ).add_to(m)           
m

## 3. Haversine방식과 KDTree방식에서 오차가 발견됨

사고지점 2번, 6번, 8번, 9번, 10번의 지도 시각화를 비교해본 결과,  
**Haversine 방식에서는 반경 300m 이내에 보호구역이 명확히 확인되었음에도 불구하고**,  
**KDTree 기반 방식에서는 동일 보호구역이 누락되는 현상**이 발생한 것을 확인할 수 있다.

이러한 차이가 **데이터 자체의 오류 때문인지, 혹은 KDTree 계산 방식의 구조적 한계로 인한 것인지**를  
정확히 확인하고자, 아래와 같은 실험 코드를 별도로 구성하였다.

In [14]:

# 기준점 설정 (서울 중심)
ref_lat, ref_lng = 37.5665, 126.9780
np.random.seed(42)
angles = np.random.uniform(0, 2*np.pi, 100)
distances = np.random.uniform(250, 350, 100)
lat_offset = distances * np.cos(angles) / 111320
lng_offset = distances * np.sin(angles) / (40075000 * np.cos(radians(ref_lat)) / 360)
zone_lats = ref_lat + lat_offset
zone_lngs = ref_lng + lng_offset

haversine_distances = [
    haversine(ref_lat, ref_lng, lat, lng) for lat, lng in zip(zone_lats, zone_lngs)
]

# KDTree 라디안 좌표 생성
ref_point_rad = (radians(ref_lat), radians(ref_lng))
zone_points_rad = list(zip(np.radians(zone_lats), np.radians(zone_lngs)))
tree = KDTree(zone_points_rad)
radian_radius = 300 / EARTH_RADIUS_M
kd_results = tree.query_ball_point(ref_point_rad, r=radian_radius)

# 결과 정리
df = pd.DataFrame({
    'lat': zone_lats,
    'lng': zone_lngs,
    'haversine_distance': haversine_distances
})
df['kd_within_300m'] = df.index.isin(kd_results)
df['haversine_within_300m'] = df['haversine_distance'] <= 300
df['difference'] = df['kd_within_300m'] != df['haversine_within_300m']
df.head()

,lat,lng,haversine_distance,kd_within_300m,haversine_within_300m,difference
0,37.564897,126.980034,252.860503,True,True,False
1,37.569183,126.976917,313.288272,False,False,False
2,37.566215,126.974831,281.121308,False,True,True
3,37.564300,126.976019,300.521046,False,False,False
4,37.568204,126.981208,340.372276,False,False,False


2번 행을 확인해보면, `haversine_distance` 값이 281.121308로 **300m 이내에 있음에도 불구하고**, `kd_within_300m` 결과는 `False`로 표시된 것을 확인할 수 있다.

이처럼 **반경 경계값(300m) 전후의 경우**, **KDTree 기반 탐색에서는 실제로 보호구역을 놓치는 사례**가 발생할 수 있으며 그 원인은 다음과 같다.

1. **KDTree는 유클리디안 거리(평면 거리)를 기반으로 한다**  
   KDTree는 공간을 **평면 좌표계로 간주**하고 거리 계산을 단순히 √(Δx² + Δy²) 방식으로 처리하기 때문에,  
   실제로 곡면인 지구 위에서의 거리(예: 위도/경도의 실제 물리적 거리)와는 차이가 발생할 수 있다.

2. **거리 계산 단위의 차이 (라디안 vs 미터)**  
   KDTree는 **라디안 거리** 단위를 사용하고, Haversine은 **지구 곡률을 고려한 실제 거리(m)** 단위를 사용한다.  
   이로 인해 특히 **위도가 높은 지역(예: 서울)**일수록 경도 1도당 거리가 줄어들며, 오차가 더욱 두드러지게 나타난다.

## 결론 : KDTree 기반 반경 필터링 로직 정밀도 보완	
이를 개선하기 위해 01_merge_protection_zone.py 코드에서 다음과 같은 2단계 판별 방식을 구현하여 적용하였다.

```python
def mark_zone_proximity(accident_df, zone_df, zone_type, radius_m=300, buffer_m =100):
    col_name = ZONE_COLUMNS[zone_type]
    accident_df[col_name] = 0

    sub_zones = zone_df[zone_df["보호구역종류"] == zone_type].copy()
    if sub_zones.empty:
        return accident_df

    # 보호구역 좌표 추출 및 라디안변환
    zone_coords_deg = sub_zones[["Y", "X"]].values
    zone_coords_rad = latlng_to_radians(sub_zones, "Y", "X")

    # 사고지점 좌표 추출 및 라디안변환
    accident_coords_deg = accident_df[["lat", "lng"]].values
    accident_coords_rad = latlng_to_radians(accident_df, "lat", "lng")

    # KDTree로 후보군 빠르게 탐색
    # 라디안을 좌표를 변환하여 KDTree 모델로 계산하는 과정에서 지도상으로는 300m에 있는 보호구역임에도
    # 인덱스가 누락되는 경우가 발생. 따라서, KDTree로는 400m 반경 이내의 후보군을 찾고, 
    # 줄어든 후보군을 haversine 거리로 재검증하여 최종적으로 300m 이내의 보호구역을 찾는 식으로 코드를 수정함
    tree = KDTree(zone_coords_rad)
    radius_rad = (radius_m + buffer_m) / EARTH_RADIUS_M
    candidates = tree.query_ball_point(accident_coords_rad, r=radius_rad)

    flags = []
    for i, match_idxs in enumerate(candidates):
        lat1, lng1 = accident_coords_deg[i]
        found = False
        for idx in match_idxs:
            lat2, lng2 = zone_coords_deg[idx]
            if haversine(lat1, lng1, lat2, lng2) <= radius_m:
                found = True
                break
        flags.append(1 if found else 0)

    accident_df[col_name] = flags
    return accident_df
```

1. **1차 후보군 필터링 (KDTree)**  
   사고지점을 중심으로 **반경 400m 이내에 존재할 가능성이 있는 보호구역을 KDTree를 통해 빠르게 탐색**한다. (후보군 확보 목적)

2. **2차 정밀 판별 (Haversine 거리)**  
   확보된 후보군에 대해서는 **지구 곡률을 반영한 Haversine 거리**를 사용하여 **실제 거리 기준 300m 이하인 보호구역만 최종적으로 판별**한다.

이러한 방식은 **성능(탐색 속도)과 정밀도(위치 정확성)를 동시에 확보**할 수 있는 절충안이며, 특히 **위도에 따른 거리 왜곡이 큰 지역에서도 안정적인 거리 판별**이 가능하다는 장점이 있다.
